In [7]:
## split -l 517317 Wales\ Away\ Raw\ Data.csv to split datasets and then rename with appendix .csv
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
# from matplotlib import pyplot as plt
# %matplotlib inline

# read in data
data1 = pd.read_csv("xaa.csv")
data2 = pd.read_csv("xab.csv")
data3 = pd.read_csv("xac.csv")

In [27]:
# to reject outliers in a list of data
def reject_outliers(data, m=3):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

# rename columns accordingly
data1.columns = ["row_id", "player_id", "player_name", "time", "elapsed_time", "lat", "lon", "speed", "heart_rate"]
data2.columns = ["row_id", "player_id", "player_name", "time", "elapsed_time", "lat", "lon", "speed", "heart_rate"]
data3.columns = ["row_id", "player_id", "player_name", "time", "elapsed_time", "lat", "lon", "speed", "heart_rate"]

newdata1 = data1.set_index('row_id')
newdata2 = data2.set_index('row_id')
newdata3 = data3.set_index('row_id')

# merge DataFrame object
frames = [data1, data2, data3]
data = pd.concat(frames)

data = pd.read_csv("Wales Away Raw Data.csv")
data = data.drop(data.columns[7], axis=1)
data.columns = ["player_name", "time", "elapsed_time", "lat", "lon", "speed", "heart_rate"]
# data.set_index('row_id')

# get list of different player IDs = 18
playerIDs = data.index.unique()

# find min and max latitude and longitude
minlat = min(data['lat'])
maxlat = max(data['lat'])
minlon = min(data['lon'])
maxlon = max(data['lon'])

print('\n The max/min lon/lat')
print(minlat)
print(maxlat)
print(minlon)
print(maxlon)

# plot the histogram of latitude and longitude data for outlier rejection
# temp = [go.Histogram(data.lat = data.lat)]
# plt.title('latitude')
# plt.show()
# data.lon.hist()
# plt.title('longitude')
# plt.show()

# # plot new histogram of longitude after outleir rejection
newtemplon = reject_outliers(data.lon)
# newtemplon.hist()
# plt.show()

# the filter values for latitude and longitude
minfilter = min(newtemplon)
maxfilter = max(newtemplon)




 The max/min lon/lat
53.2867216667
53.2942983333
-3.71859
13.58218


In [28]:
usefilter = 1

# filter out values based on outliers in lon/lat
if usefilter:
    print(len(data))
    data = data.loc[(data['lon'] <= maxfilter)]
    data = data.loc[data['lon'] >= minfilter]
    print(len(data))

# find min and max latitude and longitude
minlat = min(data['lat'])
maxlat = max(data['lat'])
minlon = min(data['lon'])
maxlon = max(data['lon'])

# scale of the latitude and longitude
latscale = 600.0/float((maxlat - minlat))
lonscale = 68.0*6.0/float((maxlon - minlon))

print(minlat)
print(maxlat)
print(minlon)
print(maxlon)
print(latscale)
print(lonscale)

1551951
1550522
53.2867216667
53.2942983333
-3.71859
-3.71261
79190.49714102653
68227.42474915771


In [31]:
# for i in range(0,1):
#     # get the player's data rows
#     playerid = playerIDs[i]
#     temp = data.loc[data.index == playerid]
    
# #     print(temp['time'])

In [ ]:
# show distribution of speed of players
# fig, axs = plt.subplots(len(playerIDs),1)
# for i in range(0,len(playerIDs)):
for i in range(0,1):
    # get the player's data rows
    playerid = playerIDs[i]
    temp = data.loc[data['player_id'] == playerid]
    
    # determine the start and end rows of first half
    firststart = temp.loc[(temp['time'] > '19:31.3')][0:1]
    firstend = temp.loc[(temp['time'] < '20:17:3')]
    firstend = firstend.reindex(index=firstend.index[::-1])[0:1]
    
    # determine the start and end rows of second half
    secondstart = temp.loc[(temp['time'] > '20:30:3')][0:1]
    secondend = temp.loc[(temp['time'] < '21:20:26')]
    secondend = secondend.reindex(index=secondend.index[::-1])[0:1]

    firststart = firststart['row_id']
    firstend = firstend['row_id']
    secondstart = secondstart['row_id']
    secondend = secondend['row_id']

    firsthalf = temp.iloc[int(firststart):int(firstend)]
    secondhalf = temp.iloc[int(secondstart):int(secondend)]
    
    
    # produce json for firsthalf
    firsthalfresponses = []
    for j in range(0, len(firsthalf)):
        # set name
        if j is 0:
            name = "Gimme"
        elif j is len(firsthalf) - 1:
            name = "Proletariat"
        else:
            name = "Along route"

        # get the longitude and latitude
        lat = firsthalf.iloc[j]['lat']
        lon = firsthalf.iloc[j]['lon']
        time = (firsthalf.iloc[j]['time']) #- initialtime

        # convert latitude 
        convertedlat = (lat-minlat)*latscale
        # convert longitude
        convertedlon = (lon-minlon)*lonscale
        
        # get heartrate
        heartrate = firsthalf.iloc[j]['heart_rate']
        
        # get speed
        speed = firsthalf.iloc[j]['speed']
        
        # json per time point
        response = {"type": "Feature",
                    "name": name,
                    "properties": {
                                "latitude": lat,
                                "longitude": lon,
                                "convertedlat": convertedlat,
                                "convertedlon": convertedlon,
                                "time": time,
                                "id": "route1",
                                "name": name,
                                "heartrate": heartrate,
                                "speed": speed
                                }
                    }

        firsthalfresponses.append(response)

    # end json description
    firsthalfjson = {"userID": playerid,
                    "type": "FeatureCollection",
                    "features": firsthalfresponses
                    }
    
    # produce json for secondhalf
    secondhalfresponses = []
    for j in range(0, len(secondhalf)):
        # set name
        if j is 0:
            name = "Gimme"
        elif j is len(secondhalf) - 1:
            name = "Proletariat"
        else:
            name = "Along route"

        # get latitude and longitude
        lat = secondhalf.iloc[j]['lat']
        lon = secondhalf.iloc[j]['lon']
        time = (secondhalf.iloc[j]['time']) #- initialtime

        # convert latitude 
        convertedlat = (lat-minlat)*latscale
        # convert longitude
        convertedlon = (lon-minlon)*lonscale

        # get heartrate
        heartrate = secondhalf.iloc[j]['heart_rate']
        
        # get speed
        speed = secondhalf.iloc[j]['speed']
        
        # json per time point
        response = {"type": "Feature",
                    "name": name,
                    "properties": {
                                "latitude": lat,
                                "longitude": lon,
                                "convertedlat": convertedlat,
                                "convertedlon": convertedlon,
                                "time": time,
                                "id": "route1",
                                "name": name,
                                "heartrate": heartrate,
                                "speed": speed
                                }
        }
        secondhalfresponses.append(response)
    
    # the end json description
    secondhalfjson = {"userID": playerid,
                    "type": "FeatureCollection",
                    "features": secondhalfresponses
                    }
    
    # saving into .json file
    firsthalf = (str(playerid) + 'firsthalf.json')
    print firsthalf
    secondhalf = (str(playerid) + 'secondhalf.json')
    print secondhalf
    %store firsthalfjson >> $firsthalf
    %store secondhalfjson >> $secondhalf   

In [ ]:
# # from __future__ import print_function
# # log = open("/Users/adam2392/Desktop/hackprinceton/test.txt", "w")
# # print("test123213123", file = log)
# %store firsthalfjson >> firsthalf.json
# %store secondhalfjson >> secondhalf.json
# # print firsthalfjson 